In [13]:
import os
import cv2
import matplotlib.pyplot as plt

In [4]:
dataset_path = '../data/hand-no-hand/'
pos_data_dir = os.path.join(dataset_path, 'positive')
neg_data_dir = os.path.join(dataset_path, 'negative') 

In [5]:
pos_img = [f for f in os.listdir(pos_data_dir) if '.jpg' in f]
neg_img = [f for f in os.listdir(neg_data_dir) if '.jpg' in f]

In [7]:
pos_img[:10]

['0.jpg',
 '1.jpg',
 '10.jpg',
 '100.jpg',
 '1000.jpg',
 '1001.jpg',
 '1002.jpg',
 '1003.jpg',
 '1004.jpg',
 '1005.jpg']

In [8]:
num_pos = len(pos_img)
threshold_pos = int(num_pos * 0.8) # 20% as test set
train_pos, test_pos = pos_img[:threshold_pos], pos_img[threshold_pos:]

In [9]:
len(train_pos), len(test_pos)

(1344, 336)

In [10]:
num_neg = len(neg_img)
threshold_neg =  int(num_neg * 0.8)
train_neg, test_neg = neg_img[:threshold_neg], neg_img[threshold_neg:]

In [21]:
X_train = []
y_train = []
for filename in train_pos:
    image = cv2.imread(os.path.join(pos_data_dir, filename), cv2.IMREAD_GRAYSCALE)
    plt.imshow(image.reshape(128, 128))
    image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    array = image.flatten()
    X_train.append(array)
    y_train.append(1)

In [22]:
for filename in train_neg:
    image = cv2.imread(os.path.join(neg_data_dir, filename), cv2.IMREAD_GRAYSCALE)
    image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    array = image.flatten()
    X_train.append(array)
    y_train.append(0)

In [44]:
X_test = []
y_test = []
for filename in test_pos:
    image = cv2.imread(os.path.join(pos_data_dir, filename), cv2.IMREAD_GRAYSCALE)
    image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    array = image.flatten()
    X_test.append(array)
    y_test.append(1)

In [45]:
for filename in test_neg:
    image = cv2.imread(os.path.join(neg_data_dir, filename), cv2.IMREAD_GRAYSCALE)
    image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    array = image.flatten()
    X_test.append(array)
    y_test.append(0)

In [46]:
len(X_train)

2688

In [47]:
len(X_test)

672

In [48]:
import numpy as np
X_train = np.array(X_train)
X_test = np.array(X_test)

In [49]:
X_train.shape

(2688, 16384)

In [50]:
X_test.shape

(672, 16384)

In [51]:
X_train = X_train.reshape(X_train.shape[0], 128, 128, 1)

In [52]:
X_train.shape

(2688, 128, 128, 1)

In [56]:
X_test = X_test.reshape(X_test.shape[0], 128, 128, 1)

In [57]:
X_test.shape

(672, 128, 128, 1)

In [34]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [58]:
print X_train[0].shape
print X_test[0].shape
print X_train.shape[0]
print X_test.shape[0]
print X_train.shape
print X_test.shape

(128, 128, 1)
(128, 128, 1)
2688
672
(2688, 128, 128, 1)
(672, 128, 128, 1)


In [59]:
X_train.shape, X_test.shape

((2688, 128, 128, 1), (672, 128, 128, 1))

In [60]:
X_train[0]

array([[[ 0.61417323],
        [ 0.69291341],
        [ 0.69685042],
        ..., 
        [ 0.68503934],
        [ 0.68110234],
        [ 0.58661419]],

       [[ 0.70078743],
        [ 0.78346455],
        [ 0.78346455],
        ..., 
        [ 0.77559054],
        [ 0.77165353],
        [ 0.67716533]],

       [[ 0.70078743],
        [ 0.78346455],
        [ 0.79133856],
        ..., 
        [ 0.77952754],
        [ 0.77559054],
        [ 0.68110234]],

       ..., 
       [[ 0.68503934],
        [ 0.65354329],
        [ 0.48425198],
        ..., 
        [ 0.17716536],
        [ 0.16929133],
        [ 0.14173228]],

       [[ 0.5905512 ],
        [ 0.54724407],
        [ 0.42519686],
        ..., 
        [ 0.16929133],
        [ 0.15354331],
        [ 0.12992126]],

       [[ 0.46062991],
        [ 0.41732284],
        [ 0.33070865],
        ..., 
        [ 0.13779527],
        [ 0.12992126],
        [ 0.11023622]]], dtype=float32)

In [73]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

In [74]:
# Hyperparameters
batch_size = 128
num_classes = 25
epochs = 10

In [75]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation = 'relu', input_shape=(128, 128, 1) ))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.20))
model.add(Dense(1, activation = 'sigmoid'))

In [76]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 126, 126, 64)      640       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 61, 61, 64)        36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 12544)             0         
__________

In [77]:
model.compile(loss = keras.losses.binary_crossentropy, 
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [78]:
history = model.fit(X_train, 
                    y_train, 
                    validation_data = (X_test, y_test), 
                    epochs=epochs, 
                    batch_size=batch_size)

Train on 2688 samples, validate on 672 samples
Epoch 1/10
2688/2688 [==============================] - 176s 65ms/step - loss: 0.5812 - acc: 0.6823 - val_loss: 0.4387 - val_acc: 0.7500
Epoch 2/10
2688/2688 [==============================] - 169s 63ms/step - loss: 0.1500 - acc: 0.9483 - val_loss: 0.2157 - val_acc: 0.9196
Epoch 3/10
 384/2688 [===>..........................] - ETA: 2:16 - loss: 0.0495 - acc: 0.9896

KeyboardInterrupt: 

In [ ]:
Train on 2688 samples, validate on 672 samples
Epoch 1/3
2688/2688 [==============================] - 174s 65ms/step - loss: 0.6364 - acc: 0.6429 - val_loss: 0.5917 - val_acc: 0.6443
Epoch 2/3
2688/2688 [==============================] - 170s 63ms/step - loss: 0.2714 - acc: 0.8977 - val_loss: 0.3056 - val_acc: 0.8705
Epoch 3/3
2688/2688 [==============================] - 180s 67ms/step - loss: 0.0915 - acc: 0.9706 - val_loss: 0.4016 - val_acc: 0.8750
('Validation accuracy:', 0.875)
Saving model